In [1]:

import time
import pandas as pd
import glob
import numpy as np
import timeit
import pickle
import ast
from datetime import datetime
from collections import Counter


from multiprocessing import Pool
from matplotlib import pyplot as plt
def toc(start_time):
    elapsed = timeit.default_timer() - start_time
    print(elapsed)
start_time = timeit.default_timer()

# import matplotlib.pyplot as plt

%matplotlib inline
# %config InlineBackend.figure_format = 'svg'
plt.rcParams.update({'font.size': 8, 'font.style': 'normal', 'font.family':'serif'})
%config InlineBackend.figure_format = 'svg'
plt.rcParams.update({'font.size': 8, 'font.style': 'normal', 'font.family':'serif'})
#matplotlib.rcParams.update({'font.size': 18})

path_dicts=


In [ ]:
start_time = timeit.default_timer()
count_store=0
t_ic=time.time()


p=Pool(processes=15)
names=path_dicts+"dict_full_careers*"

# names=path_dicts+"parquets/dict_full_careers*" # all files
files=sorted(glob.glob(names))
df_careers = pd.concat(p.map(pd.read_parquet,files),ignore_index=True)
print('loaded careers=',toc(start_time))

p.close()

#----------------------------------------------------------
### CREATING NEW COLUMNS
#----------------------------------------------------------

pd.options.mode.chained_assignment = None  # default='warn'
# df_careers["is_white"]=""
df_careers["is_white"]=np.where(df_careers["white_player"]==df_careers["player_id"],1,0)




conditions  = [ (df_careers["result_"]=='1-0') &(df_careers["is_white"]==1),
               (df_careers["result_"]=='0-1') &(df_careers["is_white"]==0),
              (df_careers["result_"]=='1/2-1/2')]
choices     = [ 1,1,0.5 ]    
df_careers["won_"] = np.select(conditions, choices, default=0)


df_careers["utcdate"]=df_careers["utcdate"]+' '+df_careers["utctime"]


#----------------------------------------------------------
### Merging datetime
#----------------------------------------------------------
from datetime import datetime
format_='%Y.%m.%d %H:%M:%S'
df_careers["utcdate"]=pd.to_datetime(pd.Series(df_careers["utcdate"]), format=format_)


#----------------------------------------------------------
### Finding career start and ends within the dataframe
#----------------------------------------------------------

ids=np.array(df_careers['player_id'])
ind=np.where(~(ids[1:ids.shape[0]]==ids[0:ids.shape[0]-1]))

ind=ind[0]
career_lens=ind[1:len(ind)]-ind[0:len(ind)-1]
tot_careers_=sum(np.array(career_lens)>500)
ind=[0]+list(ind)     


In [ ]:
start_time = timeit.default_timer()
count_store=0
t_ic=time.time()

for it_load_files in range(2,10):
    p=Pool(processes=15)
    names=path_dicts+"dict_full_careers"+str(it_load_files)+"*"

    # names=path_dicts+"parquets/dict_full_careers*" # all files
    files=sorted(glob.glob(names))
    df_careers = pd.concat(p.map(pd.read_parquet,files),ignore_index=True)
    print('loaded careers=',toc(start_time))

    p.close()

    #----------------------------------------------------------
    ### CREATING NEW COLUMNS
    #----------------------------------------------------------

    pd.options.mode.chained_assignment = None  # default='warn'
    # df_careers["is_white"]=""
    df_careers["is_white"]=np.where(df_careers["white_player"]==df_careers["player_id"],1,0)


    conditions  = [ (df_careers["result_"]=='1-0') &(df_careers["is_white"]==1),
                   (df_careers["result_"]=='0-1') &(df_careers["is_white"]==0),
                  (df_careers["result_"]=='1/2-1/2')]
    choices     = [ 1,1,0.5 ]    
    df_careers["won_"] = np.select(conditions, choices, default=0)
    df_careers["utcdate"]=df_careers["utcdate"]+' '+df_careers["utctime"]
    #----------------------------------------------------------
    ### Merging datetime
    #----------------------------------------------------------
    from datetime import datetime

    t_ic=time.time()
    format_='%Y.%m.%d %H:%M:%S'
    df_careers["utcdate"]=pd.to_datetime(pd.Series(df_careers["utcdate"]), format=format_)

    #----------------------------------------------------------
    ### Finding career start and ends within the dataframe
    #----------------------------------------------------------

    ids=np.array(df_careers['player_id'])
    ind=np.where(~(ids[1:ids.shape[0]]==ids[0:ids.shape[0]-1]))

    ind=ind[0]
    career_lens=ind[1:len(ind)]-ind[0:len(ind)-1]
    tot_careers_=sum(np.array(career_lens)>500)
    ind=[0]+list(ind)
    
    df_careers=df_careers.sort_values(['player_id', 'utcdate'])
    
    #writing to parquet
    file_name=path_dicts+"parquets/0.new_dicts_df_career"+str(it_load_files)+".csv"
    df_careers.to_csv(file_name, sep='\t',index=None)
#     names_par=path_dicts+"parquets/0.new_dicts_df_career"+str(it_load_files)+".parquet"
#     df_careers.to_parquet(names_par,index=None)

    t_oc = time.time();
    print('done=','-----',it_load_files)

        
